# Assign overlap p values to individual proteins and perform multiple testing correction

In [1]:
import pandas as pd
import numpy as np
import statsmodels.stats.multitest

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "trans"

In [3]:
overlaps = pd.read_csv(f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv", sep="\t")
pvals = pd.read_csv(f"overlap_pvals_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv", sep="\t")

## Merge in the _p_ values for each protein

We will exclude proteins that were only different in one cancer type (no overlap).

In [4]:
overlaps = overlaps[overlaps["num_cancers"] > 1].\
merge(
    right=pvals,
    left_on="num_cancers",
    right_on="overlap_size",
    how="inner"
)

## Apply multiple testing correction

In [5]:
reject, adj_pvals, alpha_sidak, alpha_bonf = statsmodels.stats.multitest.multipletests(
    pvals=overlaps["pvals"], 
    alpha=0.05, 
    method="fdr_bh"
)

overlaps = overlaps.assign(
    adj_p=adj_pvals
)

In [6]:
overlaps

,protein,cancers,mean_simp_change,num_cancers,overlap_size,pvals,adj_p
0,CNOT8,brca_luad,1,2,2,0.001777,0.001777
1,ATP6V1A,colon_luad,-1,2,2,0.001777,0.001777
2,ATP6V1E1,colon_luad,-1,2,2,0.001777,0.001777
3,ATP6V1H,colon_luad,-1,2,2,0.001777,0.001777
4,HIST1H1D,hnscc_luad,-1,2,2,0.001777,0.001777


In [7]:
overlaps.to_csv(f"sig_diff_overlaps_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv", sep="\t", index=False)